# Exam results prediction
This notebook will provide an analysis of the data and will predict the exam results of the students.
The dataset is taken from [Kaggle](https://www.kaggle.com/datasets/rkiattisak/student-performance-in-mathematics).

## Importing the libraries

In [ ]:
from os import stat
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display
import numpy as np
%matplotlib inline

## Importing the dataset

In [ ]:
# Size of the file

filename = 'data/exams.csv'

print(f'File size: {stat(filename).st_size / 1024} kB.')

# Read the data
df = pd.read_csv(filename)